## Install Prerequisites

In [1]:
! pip install -q --upgrade google-generativeai langchain langchain-community langchain-google-genai chromadb pypdf


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.3/67.3 kB 2.6 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 30.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 41.5/41.5 kB 3.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 611.1/611.1 kB 45.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.4/2.4 MB 85.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 298.0/298.0 kB 22.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 278.6/278.6 kB 25.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 94.8/94.8 kB 10.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.9/1.9 MB 75.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 94.8/94.8 kB 9.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.3/13.3 MB 99.7 MB/s eta 0:00:

## Configure the API Key

In [22]:
import os
import google.generativeai as genai
from google.colab import userdata
apikey="your api key"

genai.configure(api_key=apikey)

In [23]:
import urllib
import warnings
from pathlib import Path as p
from pprint import pprint

import pandas as pd
from langchain import PromptTemplate
from langchain.chains.question_answering import load_qa_chain
from langchain.document_loaders import PyPDFLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.vectorstores import Chroma
from langchain.chains import RetrievalQA
from langchain_google_genai import ChatGoogleGenerativeAI
warnings.filterwarnings("ignore")

In [24]:
model = ChatGoogleGenerativeAI(
    model="gemini-pro",
    google_api_key= apikey,
    temperature=0.7,
    convert_system_message_to_human=True
)

## build QnA System using PDF Data Integration

In [25]:
from pathlib import Path

file_path = Path("/content/drive/MyDrive/Oracle Notes.pdf")
if file_path.exists():
    print("File exists and is ready for processing.")
else:
    print("File does not exist. Please upload it.")

File exists and is ready for processing.


In [26]:
from langchain_community.document_loaders import PyPDFLoader

pdf_loader = PyPDFLoader("/content/drive/MyDrive/Oracle Notes.pdf")
pages = pdf_loader.load_and_split()
print(pages)

[Document(metadata={'source': '/content/drive/MyDrive/Oracle Notes.pdf', 'page': 1}, page_content='Class-1 \nTypes of data \nWe have two types of data. \n1)Unstructured Data \n2)Structured Data \n \n1)Unstructured Data \n\uf0b7 Data which is not in readable format is called unstructured data. \n\uf0b7 In general, meaning less data is called unstructured data.  \n \nex: \n 302 willorro Lakemba  SYD NSW \n  \n2)Structured Data \n\uf0b7 Data which is in readable format is called structured data.  \n\uf0b7 In general, meaning full data is called structured data. \n \nex: \n unit  street  Locality  City  State \n 302  willorro Lakemba  SYD  NSW \nClient-Server Architecture \nIn this architecture we need to see how our frontend data will store in backend.  \nDiagram: oracle1.1'), Document(metadata={'source': '/content/drive/MyDrive/Oracle Notes.pdf', 'page': 2}, page_content='FrontEnd \nThe one which is visible to the enduser to perform some operations is called \nfrontend. \nex: \n Java, Py

In [27]:
from langchain_google_genai import GoogleGenerativeAIEmbeddings
text_splitter = RecursiveCharacterTextSplitter(chunk_size=10000, chunk_overlap=1000)
context = "\n\n".join(str(p.page_content) for p in pages)
texts = text_splitter.split_text(context)
embeddings = GoogleGenerativeAIEmbeddings(model="models/embedding-001",google_api_key=apikey)

## Vector Indexing to make a Retrieval-Based QA System

In [28]:
vector_index = Chroma.from_texts(texts, embeddings).as_retriever(search_kwargs={"k":5})

qa_chain = RetrievalQA.from_chain_type(
    model,
    retriever=vector_index,
    return_source_documents=True

)

## Ask the Question and Displaying Output!

In [36]:
question = "What are cursors?"
result = qa_chain({"query": question})

# Extract and print the result
answer = result.get("result", "No answer found.")
print(f"Question: {question}\n")
print("Answer:")
print(answer)


Question: What are cursors?

Answer:
Cursors are used to iterate through a set of records in a database. They allow you to access and process the records one at a time, and you can control the order in which they are processed. This can be useful for tasks such as:

* Displaying data in a specific order
* Updating or deleting records in a specific order
* Processing data in a batch
* Iterating through a set of records that is too large to fit into memory all at once

Cursors are created using the `DECLARE` statement, and they are opened using the `OPEN` statement. The `FETCH` statement is used to retrieve the next record from the cursor, and the `CLOSE` statement is used to close the cursor when you are finished with it.

Here is an example of how to use a cursor to iterate through the records in a table:

```sql
DECLARE cursor_name CURSOR FOR
SELECT * FROM table_name;

OPEN cursor_name;

FETCH cursor_name INTO variable1, variable2, ...;

WHILE cursor_name%FOUND LOOP
    -- Do somethin